In [13]:
# Creem la conex amb la db MYSQL
import mysql.connector

db = mysql.connector.connect(
    host="localhost",
    username="root",
    password="123321",
    database="tenfe"
)

In [14]:
import pandas as pd

In [15]:
# Llegim el excel i el guardem en un dataframe
df = pd.read_excel('../data/RawData.xlsx')

df

,LINIA,ID_PARADA,HORA,DIA_SEMANA,ID_VIAJE,ORDEN
0,R1,L’Hospitalet de Llobregat,05:48:00,feiners,1,1
1,R1,Barcelona - Sants,05:55:00,feiners,1,2
2,R1,Barcelona - Pl. Catalunya,06:00:00,feiners,1,3
3,R1,Barcelona - Arc de Triomf,06:03:00,feiners,1,4
4,R1,Barcelona - El Clot Aragó,06:07:00,feiners,1,5
...,...,...,...,...,...,...
1871,R1,El Masnou,00:12:00,feiners,95,10
1872,R1,Premià de Mar,00:18:00,feiners,95,11
1873,R1,Vilassar de Mar,00:21:00,feiners,95,12
1874,R1,Cabrera de Mar - Vilassar de Mar,00:23:00,feiners,95,13


In [16]:
# Creem un cursor per poder fer les consultes
cursor = db.cursor()

In [17]:
# Extraiem el ID_PARADA del dataframe i el guardem en una llista
parades = df['ID_PARADA'].unique()

# Inserim les parades a la taula PARADES
for parada in parades:
    # Comprovem que no existeixi la parada
    cursor.execute("SELECT * FROM parades WHERE nom = %s", (parada,))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la parada
    sql = "INSERT INTO parades (nom) VALUES (%s)"
    val = (parada,)
    cursor.execute(sql, val)


# Guardem els canvis
db.commit()
df

,LINIA,ID_PARADA,HORA,DIA_SEMANA,ID_VIAJE,ORDEN
0,R1,L’Hospitalet de Llobregat,05:48:00,feiners,1,1
1,R1,Barcelona - Sants,05:55:00,feiners,1,2
2,R1,Barcelona - Pl. Catalunya,06:00:00,feiners,1,3
3,R1,Barcelona - Arc de Triomf,06:03:00,feiners,1,4
4,R1,Barcelona - El Clot Aragó,06:07:00,feiners,1,5
...,...,...,...,...,...,...
1871,R1,El Masnou,00:12:00,feiners,95,10
1872,R1,Premià de Mar,00:18:00,feiners,95,11
1873,R1,Vilassar de Mar,00:21:00,feiners,95,12
1874,R1,Cabrera de Mar - Vilassar de Mar,00:23:00,feiners,95,13


In [18]:
# Canviem del df cada nom de parada pel seu id
for _, fila in df.iterrows():
    sql = "SELECT id_parada FROM parades WHERE nom = %s"
    val = (fila['ID_PARADA'],)
    cursor.execute(sql, val)
    result = cursor.fetchone()
    # Canviem de la fila el nom de la parada pel seu id
    try:
        df.loc[df['ID_PARADA'] == fila['ID_PARADA'], 'ID_PARADA'] = result[0]
    except:
        print("error en la fila: ", fila)

df

,LINIA,ID_PARADA,HORA,DIA_SEMANA,ID_VIAJE,ORDEN
0,R1,27,05:48:00,feiners,1,1
1,R1,28,05:55:00,feiners,1,2
2,R1,29,06:00:00,feiners,1,3
3,R1,30,06:03:00,feiners,1,4
4,R1,31,06:07:00,feiners,1,5
...,...,...,...,...,...,...
1871,R1,36,00:12:00,feiners,95,10
1872,R1,37,00:18:00,feiners,95,11
1873,R1,38,00:21:00,feiners,95,12
1874,R1,39,00:23:00,feiners,95,13


In [19]:
# A tota la columna HORA la passem a string i posteriorment li treiem els ultims 3 caracters
df['HORA'] = df['HORA'].astype(str)
df['HORA'] = df['HORA'].str[:-3]

In [20]:
# Creem la linia a la taula linies si no existeix
for _, fila in df.iterrows():
    # Comprovem que no existeixi la linia
    cursor.execute("SELECT * FROM linies WHERE nom = %s", (fila['LINIA'],))
    result = cursor.fetchone()
    if result:
        continue
    # Inserim la linia
    sql = "INSERT INTO linies (nom) VALUES (%s)"
    val = (fila['LINIA'],)
    cursor.execute(sql, val)

In [21]:
# Inserim les dades de l'excel a la db
for _, row in df.iterrows():
    sql = "INSERT INTO horarios (linia, id_parada, hora, dia, id_viaje, orden) VALUES (%s, %s, %s, %s, %s, %s)"
    val = (row['LINIA'], row['ID_PARADA'], row['HORA'], row['DIA_SEMANA'], row['ID_VIAJE'], row['ORDEN'])
    cursor.execute(sql, val)

    # Mostrem el prrogres
    print(cursor.rowcount, "record inserted.")
    
db.commit()

1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record inserted.
1 record ins